In [1]:
import gymnasium as gym
from gymnasium.spaces import Discrete, Box, Sequence, Dict
import numpy as np
from src.commons import methods
import statistics

from src.rl_environments.env_impr_simp import EnvImprSimp as Env

import os

## Config

In [2]:
DEFAULT_CONFIG = {
    'alpha_range': [ i / 10 for i in range(1,10)],
    'beta_range': [round(i * 0.5, 1) for i in range(2, 6)],
    'h_range': [0.5],
    'c_range': [25],
    'total': range(10, 40), 
}

TEST_CONFIG = {
    'alpha_range': [ (3 + i / 10) for i in range(1,10)],
    'beta_range': [round(i * 0.5, 1) for i in range(2, 6)],
    'h_range': [0.5],
    'c_range': [25],
    'total': range(10, 40), 
}



In [3]:
PPO_MODEL_NAME = 'PPO_Env_Simp'

## Train PQN

In [4]:
import stable_baselines3
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv

if True:

    n_cpus = os.cpu_count()

    print(f"Number of processors: {n_cpus}")
    env = make_vec_env(lambda: Env(config=DEFAULT_CONFIG), n_envs=n_cpus, vec_env_cls=SubprocVecEnv)
    model = PPO("MlpPolicy", env, verbose=1, device='cpu')

    print(model.n_envs)

    total_timesteps = 500_000
    print(f"Training model for {total_timesteps} timesteps")
    # Start training
    model.learn(total_timesteps=total_timesteps)

    model.save(methods.file_path(PPO_MODEL_NAME, 'models'))



Number of processors: 8
Using cpu device
8
Training model for 500000 timesteps
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 3.36     |
|    ep_rew_mean     | -5.65    |
| time/              |          |
|    fps             | 11254    |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 16384    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 3.95        |
|    ep_rew_mean          | -5.29       |
| time/                   |             |
|    fps                  | 5414        |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.008861158 |
|    clip_fraction        | 0.0512      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.38  

## Generate Test Data

In [5]:
def get_realized_data(config):
    alpha = np.random.choice(config['alpha_range'])
    beta = np.random.choice(config['beta_range'])
    h = np.random.choice(config['h_range'])
    c = np.random.choice(config['c_range'])
    total = np.random.choice(config['total'])
    intervals = np.random.gamma(shape=alpha, scale=beta, size=total)
    # travel_time = sum(intervals[4:]) - np.random.exponential(scale=alpha * beta)
    travel_time = sum(intervals[4:]) - np.random.gamma(shape=4, scale=alpha*beta)
    # travel_time = np.random.gamma(shape=5, scale=alpha*beta)
    # travel_time = max(beta * 5, travel_time)
    travel_time = sum(intervals[3:]) * np.random.uniform(0, 1)
    travel_time = max(alpha * beta, travel_time)

    return alpha, beta, h, c, total, intervals, travel_time

import pandas as pd

df = pd.DataFrame(columns=['h', 'c', 'travel_time', 'total', 'intervals', 'alpha', 'beta'])

for i in range(10000):
    alpha, beta, h, c, total, intervals, travel_time = get_realized_data(TEST_CONFIG)
    row = {'h': h, 'c': c, 'travel_time': travel_time, 'total': total, 'intervals': intervals, 'alpha': alpha, 'beta': beta}
    df.loc[i] = row



## Test

In [6]:
from stable_baselines3 import PPO
model = PPO.load(methods.file_path(PPO_MODEL_NAME, 'models'))

env = Env(config=TEST_CONFIG)
rewards = {}
u_rl = {}
print(env._get_info())
for i in range(len(df)):
    row = df.iloc[i]
    state, _ = env.reset(row=row)
    done = False
    total_reward = 0
    while not done:
        action, _ = model.predict(state)  
        state, reward, done, _, info = env.step(action)
        total_reward += reward
    rewards[i] = total_reward
    u_rl[i] = env.cur_time + env.travel_time
    # print(f"Episode {i} reward: {total_reward} | h = {info['state']['h']}, c = {info['state']['c']}")
    print(f"Episode {i} reward: {total_reward} | cur_time = {info['state']['cur_time']}")

df['rewards'] = rewards
df['u_rl'] = u_rl
print(f"Average reward: {df['rewards'].mean()}")

{'hidden': {'alpha': -1, 'beta': -1, 'interval': -1, 'cum_sum_intervals': -1}, 'state': {'n': -1, 'N': -1, 'travel_time': -1, 'cur_time': -1, 'mean_n': -1, 'std_n': -1, 'u_star_hat': -1, 'last_update': -1, 'remain_ratio': -1, 'scaled_diff_u_tt': -1, 'scaled_diff_last_update': -1}}
Episode 0 reward: -10.339987107530533 | cur_time = 67.63144969232322
Episode 1 reward: -31.022877358001928 | cur_time = 27.32929618971503
Episode 2 reward: -35.806275514962195 | cur_time = 53.57900341075016
Episode 3 reward: -12.103209637057201 | cur_time = 23.96158850282745
Episode 4 reward: -11.670468460442883 | cur_time = 20.725418013344427
Episode 5 reward: -14.603557097077747 | cur_time = 98.82327672526989
Episode 6 reward: -15.026665340129433 | cur_time = 17.42895060392748
Episode 7 reward: -22.033621901336616 | cur_time = 41.75747966387794
Episode 8 reward: -18.247257994936597 | cur_time = 7.368374823597808
Episode 9 reward: -23.961405378258213 | cur_time = 19.465252086117047
Episode 10 reward: -26.934

## Add Comparison Metrics

In [7]:

from pandarallel import pandarallel

# Initialize pandarallel with progress bar enabled
pandarallel.initialize(progress_bar=True)
for i in [0, 3, 5]:
    print(f"Optimal reward at n = {i}")
    df[f'u{i}'] = df.apply(lambda row: row['intervals'][i:].sum(), axis=1)
    df[f'u_star{i}'] = df.parallel_apply(lambda row: methods.get_u_star_binary_fast(row['total'] - i, row['alpha'], row['beta'], row['h'], row['c']), axis=1)
    df[f'optimal_rewards{i}'] = df.apply(lambda row: -methods.cal_cost(row['c'], row['h'], row[f'u{i}'], row[f'u_star{i}']), axis=1)

df['direct_leave_rewards'] = df.apply(lambda row: -methods.cal_cost(row['c'], row['h'], row['intervals'][3:].sum(), row[f'travel_time']), axis=1)
df.head()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
Optimal reward at n = 0


Optimal reward at n = 3


Optimal reward at n = 5


,h,c,travel_time,total,intervals,alpha,beta,rewards,u_rl,u0,u_star0,optimal_rewards0,u3,u_star3,optimal_rewards3,u5,u_star5,optimal_rewards5,direct_leave_rewards
0,0.5,25,132.944890,28,"[5.798300475055537, 15.929158930588121, 15.753...",3.1,2.5,-10.339987,200.576340,221.256314,201.819791,-9.718261,183.774994,178.152991,-2.811002,169.457084,162.426757,-3.515164,-25.415052
1,0.5,25,25.600822,14,"[4.757629409188259, 12.088071416889893, 9.6020...",3.1,2.5,-31.022877,52.930118,114.975873,92.425283,-11.275295,88.528167,69.545907,-9.491130,76.008909,54.518799,-10.745055,-31.463672
2,0.5,25,99.644993,30,"[9.224391844794315, 5.526688755569855, 11.4671...",3.7,2.0,-35.806276,153.223997,224.836548,206.477609,-9.179469,198.618278,184.077763,-7.270258,186.097122,169.186109,-8.455506,-49.486642
3,0.5,25,15.304719,21,"[3.4487670019216803, 1.2372842379194604, 2.753...",3.7,1.0,-12.103210,39.266307,63.472726,65.526477,-25.000000,56.033266,54.945482,-0.543892,49.761473,47.946171,-0.907651,-20.364274
4,0.5,25,39.540474,11,"[12.007745455784317, 6.256821637955737, 1.7922...",3.5,2.0,-11.670468,60.265892,83.606829,62.322268,-10.642280,63.549973,42.315133,-10.617420,48.364368,29.330870,-9.516749,-12.004749


## Visualize Results

In [8]:
import plotly.graph_objects as go

all_rewards = [x for x in df.columns if 'reward' in x]

# Calculate means
means = df[all_rewards + ['h', 'c']].mean()

# Plot
fig = go.Figure(data=[go.Bar(x=means.index, y=means.values)])
fig.update_layout(title="Mean Values of Rewards, H, and C", yaxis_title="Mean Value")
fig.show()

# Calculate medians
medians = df[all_rewards + ['h', 'c']].median()

# Plot
fig = go.Figure(data=[go.Bar(x=medians.index, y=medians.values)])
fig.update_layout(title="Median Values of Rewards, H, and C", yaxis_title="Median Value")
fig.show()

In [9]:
df.head(50)

,h,c,travel_time,total,intervals,alpha,beta,rewards,u_rl,u0,u_star0,optimal_rewards0,u3,u_star3,optimal_rewards3,u5,u_star5,optimal_rewards5,direct_leave_rewards
0,0.5,25,132.944890,28,"[5.798300475055537, 15.929158930588121, 15.753...",3.1,2.5,-10.339987,200.576340,221.256314,201.819791,-9.718261,183.774994,178.152991,-2.811002,169.457084,162.426757,-3.515164,-25.415052
1,0.5,25,25.600822,14,"[4.757629409188259, 12.088071416889893, 9.6020...",3.1,2.5,-31.022877,52.930118,114.975873,92.425283,-11.275295,88.528167,69.545907,-9.491130,76.008909,54.518799,-10.745055,-31.463672
2,0.5,25,99.644993,30,"[9.224391844794315, 5.526688755569855, 11.4671...",3.7,2.0,-35.806276,153.223997,224.836548,206.477609,-9.179469,198.618278,184.077763,-7.270258,186.097122,169.186109,-8.455506,-49.486642
3,0.5,25,15.304719,21,"[3.4487670019216803, 1.2372842379194604, 2.753...",3.7,1.0,-12.103210,39.266307,63.472726,65.526477,-25.000000,56.033266,54.945482,-0.543892,49.761473,47.946171,-0.907651,-20.364274
4,0.5,25,39.540474,11,"[12.007745455784317, 6.256821637955737, 1.7922...",3.5,2.0,-11.670468,60.265892,83.606829,62.322268,-10.642280,63.549973,42.315133,-10.617420,48.364368,29.330870,-9.516749,-12.004749
5,0.5,25,16.282584,31,"[5.150043915739607, 5.198876535136254, 6.73466...",3.3,1.5,-14.603557,115.105860,144.312974,138.121267,-3.095854,127.229386,123.443395,-1.892996,118.554774,113.690547,-2.432114,-55.473401
6,0.5,25,24.348395,10,"[2.1095673190573616, 7.8167326482956065, 6.940...",3.3,2.0,-15.026665,41.777346,71.830677,51.780833,-10.024922,54.963950,33.185403,-10.889274,43.289118,21.231956,-11.028581,-15.307777
7,0.5,25,35.597798,18,"[4.360886770085802, 15.209086504361853, 10.273...",3.8,2.0,-22.033622,77.355278,121.422522,120.994043,-0.214239,91.579316,98.425400,-25.000000,83.665359,83.498317,-0.083521,-27.990759
8,0.5,25,42.737248,24,"[3.3764981036873225, 2.4872909097776934, 1.266...",3.5,1.0,-18.247258,50.105623,86.600139,71.567803,-7.516168,79.469454,61.512602,-8.978426,75.783494,54.850578,-10.466458,-18.366103
9,0.5,25,8.000000,10,"[1.524752211658509, 9.794799746350927, 7.51778...",3.2,2.5,-23.961405,27.465252,75.388063,64.431611,-5.478226,56.550722,41.413568,-7.568577,39.974650,26.581809,-6.696421,-24.275361
